## Generating close data

Importing libraries

In [1]:
from sqlalchemy import create_engine
from binance.client import Client
import pymysql
import pandas as pd
import datetime
import numpy as np
import pickle
import random

Connecting to Database

In [2]:
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

Function to fetch coin data

In [3]:
def getminutedata(symbol, start, end):
    frame = pd.DataFrame(client.get_historical_klines(symbol, Client.KLINE_INTERVAL_15MINUTE, start, end))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    frame = frame.set_index('date')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

Fetching crypto_table and id_list from database

In [4]:
cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)
full_id_list = crypto_table.id.astype(int).tolist()
full_id_list.remove(226)

Find most recent date in price table

In [5]:
dateString = "SELECT MAX(date) FROM crypto_price"
recentDate = pd.read_sql_query(dateString, engine)
resentTimestamp = recentDate.at[0, 'MAX(date)']

In [ ]:
resentTimestamp =  datetime.datetime(2022, 2, 26, 00, 00, 00)

Update Crypto Price table to current date

In [10]:
start_string = str(resentTimestamp + datetime.timedelta(minutes = 15))
end_string = datetime.datetime.now().strftime("%Y-%m-%d %H:00:00")

for id in full_id_list:
    try:
        df = getminutedata(crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0], start_string, end_string)
        df['crypto_id'] = id
    except:
        continue
    #print(df)

    df.to_sql("crypto_price", engine, if_exists="append", index=False)


In [8]:
full_id_list.index(id)

20

Updating Ticker

In [ ]:
id_update = 226
crypto_table.loc[crypto_table['id'] == id_update]['ticker']

225    RGTUSDT
Name: ticker, dtype: object